In [1]:
#%run /home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/01_funciones.ipynb
%run /Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/01_funciones.ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# cod_provincia
dict_cod_provincia = {2: "CABA",
 6: "Buenos Aires",
 10: "Catamarca",
 14: "Córdoba",
 18: "Corrientes",
 22: "Chaco",
 26: "Chubut",
 30: "Entre Ríos",
 34: "Formosa",
 38: "Jujuy",
 42: "La Pampa",
 46: "La Rioja",
 50: "Mendoza",
 54: "Misiones",
 58: "Neuquén",
 62: "Río Negro",
 66: "Salta",
 70: "San Juan",
 74: "San Luis",
 78: "Santa Cruz",
 82: "Santa Fe",
 86: "Santiago del Estero",
 90: "Tucumán",
 94: "TdF"}


dict_cod_aglomerado = {2: "Gran La Plata",
3: "Bahía Blanca - Cerri",
4: "Gran Rosario",
5: "Gran Santa Fé",
6: "Gran Paraná",
7: "Posadas",
8: "Gran Resistencia",
9: "Comodoro Rivadavia - Rada Tilly",
10: "Gran Mendoza",
12: "Corrientes",
13: "Gran Córdoba",
14: "Concordia",
15: "Formosa",
17: "Neuquén – Plottier",
18: "Santiago del Estero - La Banda",
19: "Jujuy-Palpalá",
20: "Río Gallegos",
22: "Gran Catamarca",
23: "Gran Salta",
25: "La Rioja",
26: "Gran San Luis",
27: "Gran San Juan",
29: "Gran Tucumán - Tafí Viejo",
30: "Santa Rosa – Toay",
31: "Ushuaia - Río Grande",
32: "Ciudad Autónoma de Buenos Aires",
33: "Partidos del GBA",
34: "Mar del Plata",
36: "Río Cuarto",
38: "San Nicolás – Villa Constitución",
91: "Rawson – Trelew",
93: "Viedma – Carmen de Patagones",
40: 'Resto Buenos Aires',
41: 'Resto Catamarca',
42: 'Resto Córdoba',
43: 'Resto Corrientes',
44: 'Resto Chaco',
45: 'Resto Chubut',
46: 'Resto Entre Ríos',
47: 'Resto Formosa',
48: 'Resto Jujuy',
49: 'Resto La Pampa',
50: 'Resto La Rioja',
51: 'Resto Mendoza',
52: 'Resto Misiones',
53: 'Resto Neuquén',
54: 'Resto Río Negro',
55: 'Resto Salta',
56: 'Resto San Juan',
57: 'Resto San Luis',
58: 'Resto Santa Cruz',
60: 'Resto Santa Fe',
61: 'Resto Santiago del Estero',
62: 'Resto Tucumán'}

In [3]:
aglomerado_provincias = pd.DataFrame([
["Gran La Plata", 'Buenos Aires'],
["Bahía Blanca - Cerri", 'Buenos Aires'],
["Gran Rosario", 'Sante Fe'],
["Gran Santa Fé", 'Santa Fe'],
["Gran Paraná", 'Entre Ríos'],
["Posadas", 'Misiones'],
["Gran Resistencia", 'Chaco'],
["Comodoro Rivadavia - Rada Tilly", 'Chubut'],
["Gran Mendoza", 'Mendoza'],
["Corrientes", 'Corrientes'],
["Gran Córdoba", 'Córdoba'],
["Concordia", 'Entre Ríos'],
["Formosa", 'Formosa'],
["Neuquén – Plottier", 'Neuquén'],
["Santiago del Estero - La Banda", 'Santiago del Estero'],
["Jujuy-Palpalá", 'Jujuy'],
["Río Gallegos", 'Santa Cruz'],
["Gran Catamarca", 'Catamarca'],
["Gran Salta", 'Salta'],
["La Rioja", 'La Rioja'],
["Gran San Luis", 'San Luis'],
["Gran San Juan", 'San Juan'],
["Gran Tucumán - Tafí Viejo", 'Tucumán'],
["Santa Rosa – Toay", 'La Pampa'],
["Ushuaia - Río Grande", 'TdF'],
["Ciudad Autónoma de Buenos Aires", 'CABA'],
["Partidos del GBA", 'Buenos Aires'],
["Mar del Plata", 'Buenos Aires'],
["Río Cuarto", 'Córdoba'],
["San Nicolás – Villa Constitución", 'Santa Fe'],
["Rawson – Trelew", 'Chubut'],
["Viedma – Carmen de Patagones", 'Río Negro'],
['Resto Buenos Aires', 'Buenos Aires'],
['Resto Catamarca', 'Catamarca'],
['Resto Córdoba', 'Córdoba'],
['Resto Corrientes', 'Corrientes'],
['Resto Chaco', 'Chaco'],
['Resto Chubut', 'Chubut'],
['Resto Entre Ríos', 'Entre Ríos'],
['Resto Formosa', 'Formosa'],
['Resto Jujuy', 'Jujuy'],
['Resto La Pampa', 'La Pampa'],
['Resto La Rioja', 'La Rioja'],
['Resto Mendoza', 'Mendoza'],
['Resto Misiones', 'Misiones'],
['Resto Neuquén', 'Neuquén'],
['Resto Río Negro', 'Río Negro'],
['Resto Salta', 'Salta'],
['Resto San Juan', 'San Juan'],
['Resto San Luis', 'San Luis'],
['Resto Santa Cruz', 'Santa Fe'],
['Resto Santa Fe', 'Pampeana'],
['Resto Santiago del Estero', 'Santiago del Estero'],
['Resto Tucumán', 'Tucumán'],],
columns=['Aglomerado', 'Provincia'])
aglomerado_provincias = aglomerado_provincias.set_index('Aglomerado')
map_aglomerado_provincia = aglomerado_provincias['Provincia'].to_dict()


In [4]:
def calcular_insercion_laboral(df, dict_cod_provincia, tipo='Urbano', base='Individual', confidence_level=0.95, pool=False):

    tasa, size, ratio, error = ratio_tasa_actividad(df, tipo=tipo, base=base, confidence_level=confidence_level, pool=pool)
    ratio_actividad = pd.concat([tasa, size, ratio, error], axis=1)

    tasa, size, ratio, error = trabajo_formal(df, tipo=tipo, base=base, confidence_level=confidence_level, pool=pool)
    empleo_asalariado_con_descuento = pd.concat([tasa, size, ratio, error], axis=1)

    tasa, size, ratio, error = horas_promedio_remuneradas(df, tipo=tipo, base=base, confidence_level=confidence_level, pool=pool)
    horas_remuneradas_media = pd.concat([tasa, size, ratio, error], axis=1)

    if horas_remuneradas_media['Horas promedio remuneradas'].dtype == 'object':
        horas_remuneradas_media['Horas promedio remuneradas'] = horas_remuneradas_media['Horas promedio remuneradas'].astype(float)

    insercion_laboral = (ratio_actividad['Actividad'] + empleo_asalariado_con_descuento['Formalidad'] + horas_remuneradas_media['Horas promedio remuneradas']) / 3
    insercion_laboral.name = 'Insercion laboral'

    variables = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media, insercion_laboral]
    for variable in variables:
        variable.index = variable.index.map(dict_cod_provincia)

    # Crear el DataFrame de inserción laboral
    df_insercion = pd.concat([ratio_actividad['Actividad'], empleo_asalariado_con_descuento['Formalidad'], horas_remuneradas_media['Horas promedio remuneradas'], insercion_laboral], axis=1)
    
    return df_insercion


In [5]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'

In [6]:
archivos_people_aglomerados = ['personas_T121.txt', 'personas_T221.txt', 
                               'personas_T321.txt', 'personas_T421.txt', 
                               'personas_T122.txt', 'personas_T222.txt', 
                               'personas_T322.txt', 'personas_T422.txt']

fechas_trimestrales = pd.date_range(start='2021-01-01', end='2022-12-31', freq='QS')
lista_df_insercion = []

for archivo in archivos_people_aglomerados:
    
    df = pd.read_csv(pathdata + 'data_input/' + archivo, delimiter=';', low_memory=False)
    
    df_insercion = calcular_insercion_laboral(df, dict_cod_aglomerado, tipo='Aglomerado', base='Individual', confidence_level=0.95, pool=False)
    df_insercion['PROVINCIA'] = df_insercion.index.map(map_aglomerado_provincia)
    df_insercion_provincias = df_insercion.groupby('PROVINCIA').mean()

    lista_df_insercion.append(df_insercion_provincias)    

dict_dfs_insercion_aglomerado = dict(zip(fechas_trimestrales, lista_df_insercion))

In [7]:
archivos_people_tot_urbano = ['personas_tot_urb_3T_21.txt', 
                                'personas_tot_urb_3T_22.txt']

fechas_tot_urb = pd.date_range(start='2021-07-01', end='2023-12-31', freq='AS')
lista_df_insercion = []

for archivo in archivos_people_tot_urbano:
    
    df = pd.read_csv(pathdata + 'data_input/' + archivo, delimiter=';', low_memory=False)
    
    df_insercion = calcular_insercion_laboral(df, dict_cod_provincia, tipo='Urbano', base='Individual', confidence_level=0.95, pool=False)

    lista_df_insercion.append(df_insercion)    

dict_dfs_insercion_urbano = dict(zip(fechas_tot_urb, lista_df_insercion))

In [8]:
for clave, valor in dict_dfs_insercion_aglomerado.items():
    print(clave)
    print(valor['Insercion laboral'].mean())


2021-01-01 00:00:00
84.2408672769984
2021-04-01 00:00:00
83.11054180247297
2021-07-01 00:00:00
83.24821180775747
2021-10-01 00:00:00
83.12932827619318
2022-01-01 00:00:00
83.2883915681981
2022-04-01 00:00:00
83.09436965226372
2022-07-01 00:00:00
82.75050674710972
2022-10-01 00:00:00
83.20209274269813


In [9]:
for clave, valor in dict_dfs_insercion_urbano.items():
    print(clave)
    print(valor['Insercion laboral'].mean())


2022-01-01 00:00:00
81.56606630426323


In [10]:
df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_insercion_pool = calcular_insercion_laboral(df_people_pool, dict_cod_provincia, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)

In [11]:
df_insercion['Insercion laboral'].mean()

81.02126006913399